<a href="https://colab.research.google.com/github/ynklab/cl/blob/main/semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

モデルと真理条件的意味論

NLTK: https://www.nltk.org/ は、Pythonプログラミング言語で書かれた英語の自然言語処理のためのライブラリであり、NLTKを使うことで
モデル検査や定理証明を簡単に記述できる。

In [1]:
import nltk
from nltk import *
from nltk.sem import logic
from nltk.sem import Valuation, Model
from nltk.sem.logic import LogicParser

モデルを読み込み、命題論理の意味表示の真理値をモデル検査で確かめよう。

In [2]:
v1 = """
hanako => a
jiro => b
saburo => c
smart => {a, b}
young => {a, c}
"""

v2 = """
hanako => a
jiro => b
saburo => c
smart => {a, b}
young => {b, c}
"""

In [3]:
val = Valuation.fromstring(v1)
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.sem.Assignment(dom)
m.evaluate('smart(hanako)&young(hanako)', g)

True

含意関係の証明を導出原理(resolution)に基づく自動推論で示してみよう。

In [4]:
read_expr = nltk.sem.Expression.fromstring
lf1 = read_expr('smart(hanako)&young(hanako)', type_check=True)
lf2 = read_expr('smart(hanako)', type_check=True)
ResolutionProver().prove(lf2, [lf1], verbose=True)

[1] {-smart(hanako)}  A 
[2] {smart(hanako)}   A 
[3] {young(hanako)}   A 
[4] {}                (1, 2) 



True

論理式の項と述語の型を確認してみよう。（type_check=Trueで型チェッカーがONになっている。）

In [5]:
#hanakoの型
print(lf2.argument.type)

#smartの型
print(lf2.function.type)

e
<e,?>


いま、tallの型は何も指定していないので、他の型になる可能性を考慮して、型チェッカーでは<e, ?>になっている。tallの型を明示的にシグネチャ（語彙項目）で定義して、もう一度型を確認しよう。

In [6]:
sig = {'smart': '<e, t>'}
lf2.typecheck(sig)
#tallの型
print(lf2.function.type)

<e,t>


意味辞書をシグネチャに定義して、型を確認しモデル検査や自動推論を行ってみよう。

In [7]:
sig = {'john': 'e', 'mary': 'e', 'tom': 'e', 'student': '<e,t>', 'run': '<e,t>', 'like': '<e,<e,t>>'}
examples = [r'run(john)',
             r'run(x)',
             r'john',
             r'\x.student(x)',
             r'\x.like(john,x)(mary)',
             r'like(john, mary)']
examples = [read_expr(e, type_check=True) for e in examples]
example = examples[4]
example.typecheck(sig)
print(example.type)

t


In [8]:
v1 = """
john => a
mary => b
tom => c
student => {a}
run => {a, b}
like => {(a,b),(c,b)}
"""
val = Valuation.fromstring(v1)
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.sem.Assignment(dom)

f1 = 'student(john)'
f2 = 'student(mary)'
lf1 = read_expr(f1, type_check=True)
lf2 = read_expr(f2, type_check=True)

#model checking
print(m.evaluate(f1, g))
print(m.evaluate(f2, g))

#theorem proving
ResolutionProver().prove(lf2, [lf1], verbose=True)

True
False
[1] {-student(mary)}  A 
[2] {student(john)}   A 



False

.simplify()はラムダ計算のβ簡約を行う関数である。

In [9]:
x1 = read_expr(r'\x.\y.like(x,y)(john)(mary)').simplify()
x2 = read_expr(r'like(john,mary)')
print(x1 == x2)

x1 = read_expr(r'\x.\y.like(x,y)(mary)(john)').simplify()
x2 = read_expr(r'like(john,mary)')
print(x1 == x2)

True
False
